<a href="https://colab.research.google.com/github/Evavanrooijen/AfricanGDP/blob/master/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from data import *
from model import *
from math import sqrt

NameError: ignored

In [0]:
range_N=[50]
range_T=[50]
range_alpha=[2, 5, 10]
range_a=[1, 2, 5, 10]
range_var_eps=[0.5, 1]

for var_eps in range_var_eps:
  for a in range_a:
    for alpha in range_alpha:
      for T in range_T:
        for N in range_N:
          simulation(N, T, alpha, a, var_eps)

In [0]:
def simulation(N, T, alpha, a , var_eps):
  """
  Args: DGP specification
  Returns: beaten
  """
  create_DGP(10, 50, 2, 0.5)
  T1_size = int((T*a)/(1+a))  

  RMPSE = np.zeros([N, T-T2_size])
  RMPSE_IMA = np.zeros([N, T-T2_size])

  for i in range(N):
    for t in range(T-T1_size):
      train = Y[:][:(T1_size+t)] # should be all countries, limited timehorizon
      prediction = fit_and_predict(define_parameters(train, i)) # could also return RMSPE immediately
      truth = Y[:][(T1_size+t)] # 1x1 scalar value
      RMSPE[i][t] = ((truth - prediction) / truth)**2

      # results = IMA_model.fit(trend='nc')
      fc = IMA_model.predict(train[i][T1_size+j+1])

      model_IMA = ARIMA(train, order=(0,1,1))
      model_fit = model_IMA.fit(disp=0) #trend = 'nc', or 'c'
      output = model_fit.forecast()
      prediction_IMA = output[0]
      RMSPE_IMA[i][t] = ((truth - prediction_IMA) / truth)**2
      
      
  RMSPE_model = sqrt(np.mean(RMSPE))
  RMSPE_IMA = sqrt(np.mean(RMSPE_IMA))

  beaten = compare_models(RMSPE_IMA, RMSPE_model) # over countries or time ?
  return beaten

In [0]:
def simulation(N, T, alpha, a , var_eps, runs=100):
  """
  Args: DGP specification
  Returns: beaten
  """
 
  T1_size = int((T*a)/(1+a))  
  RMSPE_model = np.zeros(runs)
  RMSPE_IMA = np.zeros(runs)

  for s in range(runs):
    Y = create_DGP(N, T, alpha, var_eps)
    RMPSE = np.zeros([N, T-T2_size])
    RMPSE_IMA = np.zeros([N, T-T2_size])

    for i in range(N):
      for t in range(T-T1_size):
        train = Y[:][:(T1_size+t)] # should be all countries, limited timehorizon
        prediction = fit_and_predict(define_parameters(train, i)) # could also return RMSPE immediately
        truth = Y[:][(T1_size+t)] # 1x1 scalar value
        RMSPE[i][t] = ((truth - prediction) / truth)**2

        # results = IMA_model.fit(trend='nc')
        fc = IMA_model.predict(train[i][T1_size+j+1])

        model_IMA = ARIMA(train, order=(0,1,1))
        model_fit = model_IMA.fit(disp=0) #trend = 'nc', or 'c'
        output = model_fit.forecast()
        prediction_IMA = output[0]
        RMSPE_IMA[i][t] = ((truth - prediction_IMA) / truth)**2
        
        
    RMSPE_model[s] = sqrt(np.mean(RMSPE))
    RMSPE_IMA[s] = sqrt(np.mean(RMSPE_IMA))

  sumBeaten, beaten = compare_models(RMSPE_IMA, RMSPE_model) # over countries or time ?
  exportResults(RMSPE_IMA, RMSPE_model, beaten)
  return print("Our model beats the benchmark {} times!".format(sumBeaten))

In [0]:
def compare_models(RMSPE_IMA, RMSPE_us):
  assert RMSPE_IMA.size == RMSPE_us.size, 'Size mismatch RMSPE arrays'
  beaten = np.zeros(RMSPE_us.size)
  for i in range(RMSPE_IMA.size):
    if (RMSPE_us[i] < RMSPE_IMA[i]):
      beaten[i] = 1
  return np.sum(beaten), beaten

In [0]:
def exportResults(RMSPE_IMA, RMSPE_model, beaten):
  export = pd.DataFrame()

  # averages over sim_runs = 100, N countries
  export['RMSPE IMA'] = RMSPE_IMA
  export['RMSPE model'] = RMSPE_model
  export['beaten'] = beaten
  export['DGP Specification'] = None

  export.to_csv('Simulation Results.csv')
  print('File is exported')

In [45]:
test2 = [ 0, 1, 2, 3]
np.array(test>test2)

array(True)

# Benchmark model

In [0]:
def recursive_estimation(i, T1, T2, steps_ahead=1):
  for j in range(T2):
    train_data = Y[i][:(T1+j)]
    model_IMA = ARIMA(train_data, order=(0, 1, 1))
    results = IMA_model.fit(trend='nc')

    fc = IMA_model.predict(Y[i][T1+j+1])
    forecasts.append(fc)
    
    RMSPE = np.sqrt(np.mean(np.square(((Y[i][T-T2:] - forecasts) / Y[i][T-T2])), axis=0))
  return RMSPE

In [0]:
# TO-DO make return RMSPE instead of MSE

from sklearn.metrics import mean_squared_error
def baseline(i):
  "function to estimate IMA using rolling window, returns MSPE"  
  train, test = T1[i], T2[i]
  history = [x for x in train]
  predictions = list()
  for t in range(len(test)):
    model = ARIMA(history, order=(0,1,1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
  MSE = mean_squared_error(test, predictions)
  return MSE